#**Práctica 5: Introducción al Procesamiento Digital de Imágenes y Visión por Computador**

Curso: Inteligencia Artificial para Ingenieros

Prof. Carlos Toro N. (carlos.toro.ing@gmail.com)

2022

**Introducción**

En esta práctica introduciremos algunos de los conceptos básicos del procesamiento digital de imágenes y visión por computador usando la [librería OpenCV](https://opencv.org/releases/).

**Dentro de los temas que veremos están:**

1. Procesamiento básico
2. Histogramas y enmascaramiento
3. Extraer características básicas
4. Detector de objetos en cascada: detección de rostros

**PREVIO:** Importaciones necesarios para la práctica

In [ ]:
import matplotlib.pyplot as plt
import cv2, matplotlib
import numpy as np

## 1. Lectura y caracterización básica de imágenes

Leyendo una imagen en OpenCV

In [ ]:
# leamos una imagen
img = cv2.imread('manzano.jpg')

# mostremos el formato de la imagen (básicamente un arreglo 3D de pixeles con información de color, en formato BGR)
print(f"La imagen tiene dimensiones {img.shape}, en estructura de dato {type(img)}\n")#la primera dimensión corresponde al ancho y la segunda al alto

# n° de canales y bits que codifican los niveles de intensidad
print(f"La imagen tiene {img.shape[-1]} canales, cada uno codificado con datos de tipo {img.dtype}\n")#notar que una imagen en escala de grises posee solo 1 canal.

# Mostremos algunos valores de la imagen
print(f"Algunos valores de los niveles de intensidad de la imagen por canal de color BGR:\n\n{img[0:3,0:3,:]}\n")

Si queremos visualizar la imagen con `matplotlib`, veremos que los colores aparecerán de forma extraña, esto se debe a que opencv lee las imágenes con los canales de color en orden BGR (Blue, Green, Red) y matplotlib espera que vengan en formato RGB. Despleguemos la imagen en ambos formatos.

In [ ]:
# transformemos la imagen a formato de color RGB
img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

# despleguemos ambas imágenes
fig = plt.figure(figsize=(6,2.5),dpi = 150)
plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Imagen en formato BGR")

plt.subplot(1,2,2)
plt.imshow(img_rgb)
#plt.axis('off')#para quitar la numeración de los ejes
plt.title("Imagen en formato RGB")
plt.show()

Una operación que puede resultar útil en algunos pre-procesamientos, es la transformación de la imagen original a escala de grises, con la función `cvtColor` podemos hacerlo. Matemáticamente, OpenCV realiza la siguiente operación: $Y_{out}=0.299*R + 0.587*G + 0.114*B$

In [ ]:
# convertimos la imagen a escala de grises
gray_img = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

# ahora, la imagen en escala de grises es solo un arreglo 2D
print(f"La imagen tiene dimensiones {gray_img.shape}, en un formato de dato {type(gray_img)}\n\n")

# despleguemos ambas imágenes
fig = plt.figure(figsize=(6,2.5),dpi = 150)
plt.subplot(1,2,1)
plt.imshow(img_rgb)
plt.title("Imagen en formato RGB")

plt.subplot(1,2,2)
plt.imshow(gray_img,cmap = "gray")
plt.title("Imagen en escala de grises")
plt.show()

**Ejercicio**: Graficar los canales de color por separado de la imagen

In [ ]:
# Código aquí

**Transformación a espacio HSV**

In [ ]:
img_hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
#visualicemos los canales por separado
fig     = plt.figure(figsize=(16,2.5),dpi = 150)
titulos = ["canal Hue", "Canal Saturation", "Canal Value"]

for i in range(3):
  plt.subplot(1,3,i+1), plt.imshow(img_hsv[:,:,i],cmap = "gray")
  plt.title(titulos[i])
  plt.xticks([]), plt.yticks([])

plt.show()

### Operaciones globales sobre la imagen

Ya que vimos que las imágenes son arreglos, podemos aplicar fórmulas o extraer descriptores estadísticos globales (un solo valor o característica que represente a la imagen completa) sobre ellas.

**A. Promedio de una imagen a color**

In [ ]:
# encontremos el promedio de la imagen a color anterior, el 'color promedio'
average_color = np.average(img_rgb,axis=(0,1))#promedio por cada canal de color

# lo anterior equivale a calcular el promedio por separado por cada canal de color y guardar el resultado en un array
average_color2= np.uint8(np.array([np.average(img_rgb[:,:,0]),np.average(img_rgb[:,:,1]),np.average(img_rgb[:,:,2])]))

# llevemos el valor decimal anterior al mismo tipo de datos de la imagen
# el color promedio será
average_color = np.uint8(average_color)
print(f"Color promedio calculado de forma compacta:  {average_color}")
print(f"Color promedio calculado de forma extendida: {average_color2}")

Formemos un pequeño arreglo con los valores calculados para desplegar el color promedio como si fuera una imagen.

In [ ]:
average_color_img = np.array([[average_color]*100]*100,np.uint8)
plt.imshow(average_color_img)
plt.show()

**B. Umbralizado y binarización de imágenes**

Experimentemos creando un umbral y binarizando (solo 2 valores) la imagen original. Con esto tendremos que para un valor umbral $u$, un pixel de imagen $p$, si $p>u$ entonces se tendrá un valor 1 y 0 para el caso contrario, o codificada en 8 bits, de 255 o 0 respectivamente. Partamos con la imagen en escala de grises, ya que es más simple entender el concepto con un solo arreglo:

In [ ]:
help(cv2.threshold)

In [ ]:
UMBRAL = 120#nivel de intensidad de la imagen que será tomado como umbral
_,img_umbralizada = cv2.threshold(gray_img,thresh = UMBRAL,maxval = 255,type = cv2.THRESH_BINARY)
img_umbralizada = cv2.cvtColor(img_umbralizada,cv2.COLOR_GRAY2RGB)

plt.figure(figsize=(6,6))
plt.imshow(img_umbralizada)
plt.show()

**C. Enmascarado de imágenes**

Usemos las imágenes binarias para enmascarar (segmentar) las zonas de interés de la imagen original:

In [ ]:
# leamos una imagen
img          = cv2.imread('Arroz.png')
img_gray     = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret_u,img_u  = cv2.threshold(img_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)# se encuentra el umbral de forma automática con el método de OTSU
img_masked   = cv2.bitwise_and(img_gray,img_u)

# despleguemos ambas imágenes
fig = plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.imshow(img_gray,cmap="gray")
plt.title("Imagen en escala de grises")

plt.subplot(1,3,2)#mascara
plt.imshow(img_u,cmap = "gray")
plt.title("Imagen binarizada")


plt.subplot(1,3,3)#imagen enmascarada
plt.imshow(img_masked,cmap = "gray")
plt.title("Imagen enmascarada")
plt.show()

##2. Histograma y enmascaramiento de una imagen

Carguemos una imagen y visualicemos su histograma

In [ ]:
img      = cv2.imread('FoggyBrooklyn.jpg')
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#histograma imagen en escala de grises
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.title("Imagen en escala de grises")
plt.imshow(img_gray, cmap ="gray")

plt.subplot(1,2,2)
plt.title("Histograma imagen en escala de grises")
plt.hist(img_gray.ravel(),bins = 256, range = [0,256])
plt.xlabel("Escala de grises"), plt.ylabel("Frecuencia estadística píxeles")
plt.show()

Observamos que la gran parte de los valores de intensidad se encuentran en los niveles de gris de alto valor, sobre 160 aprox. Ecualicemos el histograma para mejorar el contraste de la imagen:

In [ ]:
equ = cv2.equalizeHist(img_gray)

#Imagen e histograma después de ecualizarlo
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.title("Imagen con contraste mejorado")
plt.imshow(equ, cmap ="gray")

plt.subplot(1,2,2)
plt.title("Histograma ecualziado")
plt.hist(equ.ravel(),bins = 256, range = [0,256])
plt.xlabel("Escala de grises"), plt.ylabel("Frecuencia estadística píxeles")
plt.show()


##3. Procesamiento básico sobre imágenes

### Mejoramiento de bordes

In [ ]:
img = cv2.imread('ChestXray.jpg')#cargamos imagen
#Definamos kernels o filtros para mejorar la información de alta frecuencia (cambios bruscos de intensidad)
kernel1 = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])

kernel2 = np.array([[-1, -1, -1],
                    [-1, 9,-1],
                    [-1, -1, -1]])
image_sharp = cv2.filter2D(src=img, ddepth=-1, kernel=kernel2)

# despleguemos ambas imágenes
fig = plt.figure(figsize=(12,6),dpi = 150)
plt.subplot(1,2,1)
plt.imshow(img,cmap="gray")
plt.title("Imagen original")

plt.subplot(1,2,2)
plt.imshow(image_sharp,cmap = "gray")
plt.title("Imagen con bordes mejorados")

plt.show()

### Desenfoque Gaussiano

En términos matemáticos, el desenfoque Gaussiano (Gaussian blurring) se realiza aplicando a la imagen original una convolución con un filtro o kernel de la forma:

$$G(x,y)=\frac{1}{2\pi\sigma^2}e^{-\frac{x^2+y^2}{2\sigma^2}}$$

donde $\sigma$ es la desviación estándar del filtro (una medida de que tan ancho es) y $x$ e $y$ son las distancias desde el origen en la dirección del eje horizontal y vertical respectivamente.

Veamos un ejemplo, para esto carguemos una imagen:

In [ ]:
img = cv2.imread('KyotoJapan.jpg')
plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

En OpenCV podemos implementar el desenfoque Gaussiano como sigue:

In [ ]:
help(cv2.GaussianBlur)

In [ ]:
#Desenfoque Gaussiano con un kernel de 11x11
KERNEL_SIZE = (51,51)# el tamaño debe ser impar
img_blur_small = cv2.GaussianBlur(img, ksize = KERNEL_SIZE,sigmaX=0)
#cv2.imwrite('img-G-blur.jpg', img_blur_small)
plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(img_blur_small, cv2.COLOR_BGR2RGB))
plt.show()

**Algunas preguntas**
1. Experimentar cambiando el tamaño del kernel, qué observa en la imagen resultante?
2. A qué corresponde el segundo parámetro de la función, el valor de $0$ en el ejemplo anterior?
3. Definir y graficar el kernel Gaussiano en un mapa de intensidad usando la definición matemática dada. (utilice numpy y matplotlib)

##2. Ejemplo: OCR (Optical Character Recognition), Reconocimiento Óptico de Caracteres

También conocido como reconocimiento de caracteres de forma simple, tiene el objetivo la digitalización de textos, el algoritmo permite identificar automáticamente los símbolos o caracteres que son parte de un determinado alfabeto, luego, al reconocerlos, podremos usar el texto para su posterior tratamiento o edición. Usaremos en este caso tanto OpenCV como la librería [Tesseract](https://en.wikipedia.org/wiki/Tesseract_(software)) para llevar a cabo la operación, experimenta con tus propios textos!!

In [ ]:
#Instalemos la librería
!sudo apt install tesseract-ocr
!pip install pytesseract

Luego de instalar, reiniciar el entorno de ejecución

In [ ]:
import pytesseract #Importamos la librería
import matplotlib.pyplot as plt
import cv2, matplotlib

Leamos y precesemos la imagen que contiene texto

In [ ]:
img     = cv2.imread('TextoParaOCR.jpg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(6,6))
plt.title("Imagen con texto")
plt.imshow(img_rgb)
plt.show()

imprimimos el texto

In [ ]:
text = pytesseract.image_to_string(img_rgb)
print(text)

Preguntas:
1. Podremos usar la función par extraer caracteres en chino?
2. Cargar una imagen propia en letra imprenta o manuscrita y probar la metodología. Qué resultados obtuvo?
3. Experimentar con estrategias de preprocesamiento de imágenes para mejorar la exactitud de la predicción, por ej. Probar cambiando el tamaño de la imagen, umbralizando para crear una representación binaria, etc.

##3. Capturar una imagen desde webcam y detección de rostros
Capturaremos una imagen desde la webcam y ejecutaremos detección de rostro sobre esta imagen capturada. La predicción de rostros en la imagen adquirida usando el clasificador en Cascada Haar pre-entrenado y disponible en OpenCV.

**Detección de Rostros:**

**Previo**: Ejecutar la siguiente función para capturar una imagen desde la webcam usando google colab (la proporciona la misma plataforma). Para código ejecutado de forma local en sus PCs, simplemente usar las funciones disponibles en OpenCV (mucho más simple, usar `VideoCapture()`)

In [ ]:
#@title Captura una imagen desde webcam: take_photo()
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode , b64encode

def take_photo(filename='photo.jpg', quality=0.8): # por defecto, la imagen capturada se llamará photo.jpg
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data)
  #print(gray.shape)
  # save image
  cv2.imwrite(filename, img)

  return filename

  #function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

In [ ]:
import numpy as np
# Inicializamos el detector de rostros
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

# realicemos una predicción sobre la imagen capturada
try:
  filename = take_photo('photo.jpg')#capturamos imagen
  img      = cv2.imread(filename)

# Obtener las coordenadas de los rostros detectados con detector Cascada Haar
  faces = face_cascade.detectMultiScale(img)
# dibujar caja rectangular en rostro detectado
  for (x,y,w,h) in faces:
      img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

  # Mostrar la imagen capturada y con los rectángulos que corresponda.
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.show
except Exception as err:
  # except se ejecuta solo si se producen errores en try:
  # útil, yaque a veces al trabajar con dispositvos como webcam,
  # se puede desconectar o generar un error al querer conectar.
  print(str(err))

## Procesamiento de un video (frame a frame)

**Previo**: Ejecutar las siguientes funciones útiles para desplegar video en Colab

In [ ]:
#@title Para desplegar un video
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML

def display_video(video, SIZE = (6,6)):
    fig = plt.figure(figsize=SIZE)  #Display size specification

    mov = []
    for i in range(len(video)):  #Append videos one by one to mov
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])

    #Animation creation
    anime = animation.ArtistAnimation(fig, mov, interval=50, repeat_delay=1000)

    plt.close()
    return anime

In [ ]:
# Instalación necesario de codec
!pip install imageio-ffmpeg

Subir video_sample.mp4 ([fuente](https://www.youtube.com/watch?v=njOndS-e0cw)) que se entregó con la práctica y luego ejecutar el siguiente código para desplegarlo:

In [ ]:
video = imageio.mimread('video_sample.mp4',memtest=False)  #Cargamos video
#video = [resize(frame, (256, 256))[..., :3] for frame in video]    #Ajuste de tamaño (si fuera necesario)
HTML(display_video(video, SIZE = (6,4)).to_html5_video())  #Despliegue del video en linea en HTML5

Como ejemplo para guardar un video, ejecturemos un procesamiento sobre cada frame del video y lo guardaremos en un archivo `video_procesado.mp4` que podremos descargar después.

In [ ]:
cap = cv2.VideoCapture("video_sample.mp4")
#Información del video original
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
FPS    = float(cap.get(cv2.CAP_PROP_FPS))# frame rate del video original

# Definimos el codec y creamos el objeto VideoWriter() con el nombre del video a guardar
fourcc = cv2.VideoWriter_fourcc(*'mp4v') #codec
out    = cv2.VideoWriter('video_procesado.mp4', fourcc, FPS, (width,  height), False)# False indica que no escribiremos frames a color
                                                                                     # Cambiar a True si la salida serán frames a color
count = 0
# Ejecutamos el procesamiento por cada frame y guardamos frame a frame el nuevo video
while cap.isOpened():

    ret, frame = cap.read()# leemos frame a frame

    if not ret:# en caso de que no queden más frames en el video, terminamos el ciclo
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # Procesamiento: flip y detección de bordes *****
    frame  = cv2.flip(frame, 1)#flip horizontal
    frame  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)# pasamos a escala de gris para facilitar procesamiento
    frame  = cv2.Canny(frame,threshold1=20, threshold2=150)

    count +=1# contamos total de frames procesados
    # escribimos el frame procesado
    out.write(frame)
    if cv2.waitKey(1) == ord('q'):
        break

print(f"Se procesaron un total de {count} frames")
# Liberamos objetos de memoria cuando se termine el loop
cap.release()
out.release()
cv2.destroyAllWindows()

Despleguemos el video procesado

In [ ]:
video = imageio.mimread('video_procesado.mp4',memtest=False)  #Loading video
HTML(display_video(video, SIZE = (6,4)).to_html5_video())  #Inline video display in HTML5

## Detección de rostros, ejemplo script con webcam

Abrir con algún IDE que soporte Python, script `Detecta_Rostros_Webcam.py`. Requiere que instalen OpenCV. En Anaconda, instalarlo desde el Anaconda Prompt con el siguiente comando o directo en el IDE: `pip install opencv-python`

##**Ejercicios**

1. Implementar el detector en cascada pero usandolo para detectar autos, en internet existen modelos pre-entrenados para openCV que implementan esta solución. Usar el video adjunto `Traffic.mp4` o uno propio para probarlo.

2. Cargar y procesar un video. Guardar un video que contenga el video original, al lado superior derecho la componente R de los frames, abajo a la izquierda, el canal G, y abajo a la derecha el canal B.  

3. En el ejemplo visto de detección de rostros usando webcam (archivo `Detecta_Rostros_Webcam.py`), al llamar al detector de rostros para realizar predicciones `face_cascade.detectMultiScale(gray, 1.1, 4)`, ¿qué representan el primer y segundo parámetros con valores `1.1` y `4` respectivamente?

## Referencias

**1.**  Detectores en Cascada con OpenCV: [enlace](https://pyimagesearch.com/2021/04/12/opencv-haar-cascades/).

OTROS EJEMPLOS


- OpenCV OCR + Tesseract: https://pyimagesearch.com/2018/09/17/opencv-ocr-and-text-recognition-with-tesseract/
- OCR con Keras-OCR: https://www.analyticsvidhya.com/blog/2022/09/extract-text-from-images-quickly-using-keras-ocr-pipeline/